# 第6章多次元データの解析

# 6.1sklearnの手書き数字

In [ ]:
from sklearn.datasets import load_digits
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# 手書き数字のロード
digits = load_digits()

In [ ]:
print(digits['DESCR'])

In [ ]:
len(digits)

In [ ]:
digits

In [ ]:
len(digits.data)

In [ ]:
len(digits.target)

In [ ]:
len(digits.images)

In [ ]:
#最初の 10 個の画像をみる
for label,img in zip(digits.target[:10],digits.images[:10]):
     plt.subplot(2,5,label+1)
     plt.axis('off')
     plt.imshow(img,cmap=plt.cm.gray_r, interpolation='nearest')
     plt.title('Digit: {}'.format(label))

In [ ]:
#offset から10個の画像を表示
offset=1000
for i in range(10):
  img = digits.data[offset+i].reshape(8,8)
  plt.subplot(2,5,i+1)
  plt.axis('off')
  plt.imshow(img,cmap=plt.cm.gray_r,
  interpolation='nearest')
  plt.title('Digit: {}'.format(digits.target[offset+i]))

問題 6.1.1

In [ ]:
d_mean = []
d_len = []
for i in range(10):
  flag = (digits.target == i)
  d = digits.data[flag]
  print('{}の個数: {}'.format(i,len(d)))
  d_mean.append(sum(d)/len(d))
  d_len.append(len(d))
d_mean = np.array(d_mean) # list -> np.array
d_len = np.array(d_len)

In [ ]:
for i in range(10):
  img = d_mean[i].reshape(8,8)
  plt.subplot(2,5,i+1)
  plt.axis('off')
  plt.imshow(img,cmap=plt.cm.gray_r,interpolation='nearest')
  plt.title('Digit: {}'.format(i))

# 6.2 主成分分析

In [ ]:
import numpy as np
import pandas as pd

X1 = np.array([50,80,100,0,0])
X2 = np.array([50,80,100,0,0])
X3 = np.array([50,0,100,80,0])
X4 = np.array([50,0,100,80,0])

In [ ]:
dic = {'Jp': X1, 'Eng': X2, 'Math':X3, 'Sci':X4}
df = pd.DataFrame(dic,index=['A','B','C','D','E'])

# 合成変量の y1, y2 を作るための初期化
dfy = pd.DataFrame(index = df.index, columns=['y1','y2'])

In [ ]:
# df の値の表示
print(df)

In [ ]:
#dfy に値を代入
dfy['y1'] = (df["Jp"]+ df["Eng"]+df["Math"]+df["Sci"])/2
dfy['y2'] = (df["Jp"]+ df["Eng"]-df["Math"]-df["Sci"])/2

print(dfy)

In [ ]:
# y1, y2 の相関係数の計算 (0である)

dfy.corr()

In [ ]:
dfy.cov()

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
pca = PCA(n_components=2)
pca.fit(df)

# 主成分(ベクトル)
print(pca.components_)

In [ ]:
# 第一主成分 a_1 と第二主成分 a_2 が得られている

# 主成分の固有値
pca.explained_variance_

In [ ]:
# y_1 と y_2 の分散の値が得られている

In [ ]:
pca.explained_variance_ratio_

In [ ]:
np.cumsum(pca.explained_variance_ratio_)

In [ ]:
df.cov()

In [ ]:
dS=pca.transform(df)
print(dS)

In [ ]:
plt.plot(dS[:,0],dS[:,1],'ko')
for i in range(len(df.index)):
  plt.annotate(df.index[i],(dS[i,0],dS[i,1]))

# 6.2.1 手書き数字の平均画像の主成分分析

In [ ]:
from sklearn.decomposition import PCA
import numpy as np
import matplotlib.pyplot as plt

pca = PCA(n_components=2)
pca = pca.fit(d_mean)
pca_d_mean = pca.transform(d_mean)
label=np.array(range(10)) #数字のラベル用

# 主成分をプロットする
plt.scatter(pca_d_mean[:,0],
pca_d_mean[:,1],c=label)
plt.title('principal component')
plt.xlabel('pc1')
plt.ylabel('pc2')
for i in range(10):
  plt.annotate(i,(pca_d_mean[i,0],pca_d_mean[i,1]))

In [ ]:
# 主成分の寄与率を出力する
print(' 各次元の寄与率: {0}'.format(
np.round(pca.explained_variance_ratio_,3)))
print(' 累積寄与率: {0}'.format(
np.round(np.cumsum(pca.explained_variance_ratio_),3)))

In [ ]:
pcaAll = PCA().fit(d_mean)
print('各次元の寄与率: {0}'.format(np.round(
pcaAll.explained_variance_ratio_,3)))
print('累積寄与率: {0}'.format(np.round(np.cumsum(pcaAll.explained_variance_ratio_),3)))

# 6.2.2 2 次元の平均画像データに k-means 法を適用

In [ ]:
from sklearn.cluster import KMeans
#クラス数を 3 として k-means 法を適用
kmeans = KMeans(n_clusters=3)
kmeans.fit(pca_d_mean)
y_kmeans = kmeans.predict(pca_d_mean)

plt.scatter(pca_d_mean[:, 0], pca_d_mean[:, 1], c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black', s=50, alpha=0.5);
for i in range(10):
  plt.annotate(i,(pca_d_mean[i,0],pca_d_mean[i,1]))

# 6.3 多次元尺度構成法

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import MDS

#駅間の距離
D2=[[0.0, 3.6, 15.8, 30.3, 18.2, 28.8],
 [3.6, 0.0, 12.2, 26.7, 21.8, 32.4],
 [15.8, 12.2, 0.0, 14.5, 34.0, 36.6],
 [30.3, 26.7, 14.5, 0.0, 48.5, 59.1],
 [18.2, 21.8, 34.0, 48.5, 0.0, 10.6],
 [28.8, 32.4, 36.6, 59.1, 10.6, 0.0]]

eki_label=["Tokyo","Ueno","Kagaguchi","Ohmiya","Kawasaki","Yokohama"]

 #MDS には距離の 2 乗を用いる.
D22=np.array(D2)**2
print(D22)
D2=D22


In [ ]:
mds_model = MDS(n_components=2,dissimilarity='precomputed')
transformed = mds_model.fit_transform(D2)
plt.scatter(transformed[:, 0], transformed[:, 1])
for i in range(len(eki_label)):
  plt.annotate(eki_label[i],(transformed[i, 0],transformed[i, 1]))
plt.axis('equal');

# 6.3.1多次元尺度構成法を手書き数字の平均画像へ適用

In [ ]:
mds_model = MDS(n_components=2)
transformed = mds_model.fit_transform(d_mean)
plt.scatter(transformed[:, 0], transformed[:, 1])
for i in range(10):
  plt.annotate(i,(transformed[i, 0],transformed[i, 1]))
plt.axis('equal');

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3)
kmeans.fit(transformed)
y_kmeans = kmeans.predict(transformed)

plt.scatter(pca_d_mean[:, 0], pca_d_mean[:, 1],
c=y_kmeans, s=50, cmap='viridis')
centers = kmeans.cluster_centers_
plt.scatter(centers[:, 0], centers[:, 1], c='black',
s=50, alpha=0.5);
for i in range(10):
  plt.annotate(i,(pca_d_mean[i,0],pca_d_mean[i,1]))

# 6.4 階層的クラスタリング

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.cluster.hierarchy import dendrogram, linkage

In [ ]:
Z = linkage(d_mean, method='ward', metric="euclidean")
dendrogram(Z, labels=range(10))
dendrogram(Z)
plt.title('Clustering of means of handwritten data')
plt.show()

In [ ]:
Z

In [ ]:
#最短距離法によるクラスタリング
Zs = linkage(d_mean, method='single', metric="euclidean")
dendrogram(Zs, labels=range(10))
plt.title('Clustering of means of handwitten data')
plt.show()

Zs